In [ ]:
pip install qiskit[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.3/241.3 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 KB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram, plot_state_qsphere
from qiskit.quantum_info import Statevector, random_statevector

In [ ]:
oracle_type = input('Would you like to test using a constant or balanced oracle? ')
oracle_type = oracle_type.lower()
oracle_size = int(input('How many input arguments should this oracle function take in? '))

Would you like to test using a constant or balanced oracle? Balanced
How many input arguments should this oracle function take in? 2


In [ ]:
def measure_and_report(qc, oracle_size):
  # for i in range(oracle_size+1):
    # qc.measure(i, i)
  qc.measure(0, 0)
  qc.measure(1, 0)
  qc.measure(2, 0)

  simulator = QasmSimulator()

  job = simulator.run(qc)
  result = job.result()
  counts = result.get_counts(qc)

  return counts

In [ ]:
def apply_hadamard_gates(qc, oracle_size):
  for i in range(oracle_size+1):
    qc.h(i)
  return qc

In [ ]:
test = Statevector.from_label('000')

main_circuit = QuantumCircuit(oracle_size + 1)
main_circuit.x(oracle_size)
test.evolve(main_circuit)

Statevector([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j],
            dims=(2, 2, 2))


In [ ]:
main_circuit = apply_hadamard_gates(main_circuit, 2)
print(main_circuit)

     ┌───┐     
q_0: ┤ H ├─────
     ├───┤     
q_1: ┤ H ├─────
     ├───┤┌───┐
q_2: ┤ X ├┤ H ├
     └───┘└───┘
